# OS11 - Exercises 0-6

## Exercise 0

Build a function that transforms a Node-Node matrix to a Node-Arc matrix.

### Data Introduction

Data will be introduced on a list of (origin, dest, cost) tuples.

In [1]:
datos_originales = []

In [ ]:
datos_originales.append(())